<a href="https://colab.research.google.com/github/niksisons/image_processing/blob/main/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82%D0%BD%D0%B0%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0_%E2%84%965_%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D0%B8_%D1%81%D1%80%D0%B0%D0%B2%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B4%D0%B5%D1%82%D0%B5%D0%BA%D1%82%D0%BE%D1%80%D0%BE%D0%B2_%D0%BA%D0%BB%D1%8E%D1%87%D0%B5%D0%B2%D1%8B%D1%85_%D1%82%D0%BE%D1%87%D0%B5%D0%BA_%D0%B8_%D0%B4%D0%B5%D1%81%D0%BA%D1%80%D0%B8%D0%BF%D1%82%D0%BE%D1%80%D0%BE%D0%B2_%D0%B2_OpenCV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Проектная работа №5. Анализ и сравнение детекторов ключевых точек и дескрипторов в OpenCV**


## **1. Цель работы**



Выполнить экспериментальное исследование детекторов и дескрипторов ключевых точек (Harris, FAST, ORB, SIFT, AKAZE, BRISK и др.) в различных условиях и для разных типов изображений, оценить их качество и производительность по заданным метрикам и сформулировать рекомендации по выбору оптимальных комбинаций «детектор + дескриптор» для конкретных задач.

## **2. Задачи работы**




1. **Сравнить** работу детекторов и дескрипторов в экстремальных условиях (размытие, шум, сжатие и др.) и оценить их устойчивость, точность локализации и скорость (Задание 1).
2. **Исследовать** мультимодальное сопоставление изображений (RGB/IR, день/ночь, спутник/карта и др.) и оценить качество сопоставлений по метрикам Precision/Recall, F1, ROC-AUC (Задание 2).
3. **Оценить** поведение детекторов/дескрипторов в специфических доменах (медицина, текстуры, документы) по профильным метрикам (регистрация, различимость текстур, качество OCR) (Задание 3).
5. **Собрать и проанализировать** результаты в виде таблиц, графиков и визуализаций, **сделать выводы** и **дать рекомендации** по выбору методов.

---






## **3. Подготовка рабочей среды и данных**



### **3.1. Настройка окружения**

**Что нужно сделать:**

- Установить/проверить наличие библиотек:
  - `opencv-python` (cv2)
  - `numpy`
  - `matplotlib`
  - при необходимости для метрик классификации — `scikit-learn` (можно обойтись и без него, но он упрощает построение PR/ROC).
- Настроить рабочую среду (Jupyter Notebook / Google Colab / локальный Python-скрипт).
- Собрать набор тестовых изображений (из интернета):
  - натурные сцены с текстурами (здания, природные объекты, городские сцены),
  - сцены с текстом (документы, вывески),
  - текстурные поверхности (ткани, дерево, бетон),
  - пары для мультимодальных задач (день/ночь, спутник/карта, RGB/IR — при наличии),
  - при желании — медицинские изображения (рентген, МРТ и т.п.).




In [ ]:
# Ваш код
# Базовые импорты

---




### **3.2. Базовые функции для работы с изображениями**

**Что нужно реализовать:**

1. **Функции загрузки** изображений с диска по пути (с возможным приведением к одному размеру при необходимости).
2. **Функцию визуализации** одного или нескольких изображений в виде сетки (matplotlib).
3. (Опционально) Функцию перевода в оттенки серого и нормализации.


In [ ]:
# Ваш код
# Функции:
# - load_image(path)
# - show_images(list_of_images, titles=None, cols=...)

---

### **3.3. Базовые функции измерения времени**

**Что нужно реализовать:**

- Простую функцию-обёртку для замера времени выполнения произвольной операции (например, детекция и описание ключевых точек).

**Пояснение:**  
Эта функция будет использоваться в Задании 1 для измерения производительности (время на одно изображение и FPS).



In [ ]:
# Ваш код

# Функция:
# - measure_time(func, *args, **kwargs) -> (result, elapsed_time)

## **4. Задание 1. Сравнительный анализ детекторов в экстремальных условиях**



### **4.1. Формирование датасета с искажениями**





**Цель:**  
Проверить устойчивость детекторов/дескрипторов к различным искажениям изображения.

**Что нужно сделать:**

1. Выбрать **несколько базовых изображений** (3–5 штук) с богатыми текстурами и структурой:
   - городские сцены (здания),
   - природные сцены (лес, горы),
   - технические объекты (машины, здания с повторяющимися элементами).

2. Для каждого базового изображения сгенерировать **варианты с искажениями**:
   - **Размытие движения (motion blur)** — свёртка с линейным фильтром.
   - **Изменение освещения**:
     - затемнение / осветление (умножение на коэффициент, добавление смещения),
     - имитация «день/ночь» (снижение яркости, усиление контраста).
   - **Поворот и масштабирование** (cv2.warpAffine / cv2.resize).
   - **Шум**:
     - гауссовский (добавление случайной компоненты),
     - импульсный («соль и перец»).
   - **Сжатие JPEG** с разным качеством (cv2.imwrite с параметром `cv2.IMWRITE_JPEG_QUALITY`).

3. Для части искажений, где это возможно (поворот, масштаб, аффинное/проективное преобразование), **сохранять или вычислять известное преобразование** (матрицу аффинного преобразования или гомографии), чтобы использовать её для расчёта метрик (повторяемость, точность локализации).

In [ ]:
# Ваш код

# Функции для аугментации:
#
# - apply_motion_blur(image, kernel_size, direction=...)
# - change_brightness_contrast(image, alpha, beta)
# - add_gaussian_noise(image, sigma)
# - add_salt_pepper_noise(image, amount)
# - jpeg_compress_decompress(image, quality)


---



### **4.2. Реализация пайплайна «детектор + дескриптор + сопоставление»**



**Что нужно сделать:**

1. Выбрать набор комбинаций:
   - SIFT (детектор+дескриптор),
   - ORB (детектор+дескриптор),
   - AKAZE (детектор+дескриптор),
   - FAST + SIFT (быстрый детектор, «тяжёлый» дескриптор),
   - BRISK (детектор+дескриптор) — при желании.

2. Для каждой комбинации реализовать:
   - детекцию ключевых точек на **оригинальном** и **искажённом** изображениях;
   - вычисление дескрипторов;
   - сопоставление (например, BFMatcher c расстоянием Хэмминга для бинарных дескрипторов и L2 для вещественных).



In [ ]:
# Ваш код (См. теорию)
# - detect_and_describe(image, method="SIFT"/"ORB"/"AKAZE"/"FAST+SIFT")
# - match_descriptors(desc1, desc2, method="BF"/"FLANN")

---




### **4.3. Метрики для оценки**

#### **4.3.1. Повторяемость (repeatability)**

**Смысл метрики:**  
Показывает, какой процент ключевых точек, найденных на одном изображении, можно «узнать» на другом при известном геометрическом преобразовании (аффинном или гомографии).

**Идея:**

1. Имеем:
   - набор ключевых точек `kp1` на изображении 1,
   - набор ключевых точек `kp2` на изображении 2,
   - известную матрицу преобразования `H` (гомография или аффинное преобразование).
2. Преобразуем координаты `kp1` в систему координат второго изображения.
3. Считаем, что точка повторилась, если в окрестности радиуса `r` от преобразованной точки есть ключевая точка из `kp2`.
4. Повторяемость =  
   $
   \text{repeatability} = \frac{N_{\text{совпавших}}}{\min(N_1, N_2)}
   \times 100\%
   $

```python
import numpy as np
import cv2

def keypoints_to_array(keypoints):
    """
    Преобразует список cv2.KeyPoint в массив координат shape (N, 2).
    """
    return np.array([kp.pt for kp in keypoints], dtype=np.float32)

def transform_points(points, H):
    """
    Преобразует точки (x, y) матрицей гомографии H.
    
    points: np.array shape (N, 2)
    H: np.array shape (3, 3)
    """
    num = points.shape[0]
    # Добавляем однородную координату
    pts_h = np.hstack([points, np.ones((num, 1), dtype=np.float32)])
    pts_t = (H @ pts_h.T).T   # shape (N, 3)
    # Переход обратно к декартовым координатам
    pts_t = pts_t[:, :2] / pts_t[:, 2:3]
    return pts_t

def compute_repeatability(kp1, kp2, H_1_to_2, radius=3.0, image_shape_2=None):
    """
    Вычисляет повторяемость ключевых точек.
    
    kp1, kp2: списки cv2.KeyPoint для изображений 1 и 2.
    H_1_to_2: матрица гомографии (из 1 в 2).
    radius: допустимое отклонение (в пикселях) для совпадения точки.
    image_shape_2: (h, w) изображения 2 для ограничения области (опционально).
    """
    if len(kp1) == 0 or len(kp2) == 0:
        return 0.0, 0
    
    pts1 = keypoints_to_array(kp1)  # (N1, 2)
    pts2 = keypoints_to_array(kp2)  # (N2, 2)
    
    # Преобразуем точки из изображения 1 в систему координат 2-го
    pts1_t = transform_points(pts1, H_1_to_2)  # (N1, 2)
    
    # Опционально отбрасываем точки, которые вышли за границы 2-го изображения
    if image_shape_2 is not None:
        h2, w2 = image_shape_2[:2]
        mask_inside = (
            (pts1_t[:, 0] >= 0) & (pts1_t[:, 0] < w2) &
            (pts1_t[:, 1] >= 0) & (pts1_t[:, 1] < h2)
        )
        pts1_t = pts1_t[mask_inside]
    
    if pts1_t.shape[0] == 0:
        return 0.0, 0
    
    # Для поиска ближайших точек можно использовать простой перебор
    # (для учебных размеров этого достаточно).
    matched = 0
    for p in pts1_t:
        dists = np.linalg.norm(pts2 - p[None, :], axis=1)
        if np.min(dists) <= radius:
            matched += 1
    
    N1 = pts1_t.shape[0]
    N2 = pts2.shape[0]
    denominator = max(1, min(N1, N2))
    
    repeatability = matched / denominator * 100.0
    return repeatability, matched
```


---

#### **4.3.2. Точность локализации (mean localization error)**



**Смысл метрики:**  
Если у нас есть пары «соответствующих» точек, можно измерить среднее расстояние между реальным положением точки и предсказанным (по гомографии). Чем меньше ошибка, тем лучше локализация.

**Идея (упрощённо):**

- Для каждой пары соответствий `(p1, p2)`:
  - преобразуем `p1` матрицей `H` → `p1’`,
  - считаем расстояние `d = ||p1’ − p2||`,
- Средняя ошибка — среднее значение `d` по всем «хорошим» соответствиям (например, инлайерам RANSAC).




```python
def compute_localization_error(matches, kp1, kp2, H_1_to_2, inlier_mask=None):
    """
    Вычисляет среднюю ошибку локализации по сопоставлениям.
    
    matches: список cv2.DMatch (обычно после фильтрации и RANSAC).
    kp1, kp2: списки cv2.KeyPoint для изображений 1 и 2.
    H_1_to_2: матрица гомографии.
    inlier_mask: опциональная маска инлайеров (список или np.array из 0/1).
    """
    if len(matches) == 0:
        return None
    
    pts1 = []
    pts2 = []
    for i, m in enumerate(matches):
        if (inlier_mask is not None) and (not inlier_mask[i]):
            continue
        pts1.append(kp1[m.queryIdx].pt)
        pts2.append(kp2[m.trainIdx].pt)
    
    if len(pts1) == 0:
        return None
    
    pts1 = np.array(pts1, dtype=np.float32)
    pts2 = np.array(pts2, dtype=np.float32)
    
    pts1_t = transform_points(pts1, H_1_to_2)
    errors = np.linalg.norm(pts1_t - pts2, axis=1)
    return float(np.mean(errors)), errors
```

---



#### **4.3.3. Время выполнения и FPS**

**Смысл:**  
Оценить, сколько времени занимает полный пайплайн (детекция + дескрипторы + сопоставление) на одном изображении и как это зависит от разрешения.

**Идея:**

- Для каждой комбинации (метод, разрешение) измеряем среднее время по нескольким запускам.
- FPS ≈ 1 / (среднее время кадра).

```python
import time

def measure_pipeline_time(pipeline_func, image1, image2, runs=5):
    """
    Измеряет среднее время выполнения пользовательского пайплайна.
    
    pipeline_func: функция, внутри которой выполняются
                   детекция, вычисление дескрипторов и сопоставление.
    image1, image2: пара изображений.
    """
    times = []
    for _ in range(runs):
        start = time.time()
        _ = pipeline_func(image1, image2)
        times.append(time.time() - start)
    mean_time = np.mean(times)
    fps = 1.0 / mean_time if mean_time > 0 else None
    return mean_time, fps, times
```


---

#### **4.3.4. Процент правильных сопоставлений и устойчивость к выбросам (RANSAC inliers)**





**Смысл:**

- **Процент правильных сопоставлений** — доля инлайеров (правильных соответствий) среди всех найденных сопоставлений.
- **Устойчивость к выбросам** — насколько хорошо метод справляется с ложными сопоставлениями (высокий процент инлайеров = хорошая устойчивость).

**Типичный сценарий:**

1. Получили список сопоставлений `matches`.
2. Построили гомографию через `cv2.findHomography(..., method=cv2.RANSAC, ...)`, получили маску инлайеров `mask`.
3. Считаем процент инлайеров.

```python
def compute_inlier_ratio(matches_mask):
    """
    Вычисляет долю инлайеров по маске, возвращаемой cv2.findHomography.
    
    matches_mask: np.array или список 0/1 (или True/False),
                  длина равна числу сопоставлений, поданных в findHomography.
    """
    mask = np.array(matches_mask).astype(bool)
    total = len(mask)
    if total == 0:
        return 0.0, 0
    inliers = np.sum(mask)
    ratio = inliers / total * 100.0
    return ratio, inliers
```

---

### **4.4. Проведение экспериментов и сбор результатов**





**Что нужно сделать:**

1. Для каждой комбинации детектор+дескриптор (SIFT, ORB, AKAZE, FAST+SIFT, BRISK и т.п.) и каждого типа искажения:
   - измерить **повторяемость**;
   - измерить **точность локализации** (при наличии гомографии);
   - измерить **время выполнения** и **FPS**;
   - вычислить **процент инлайеров (inlier ratio)** после RANSAC.

2. Свести результаты в таблицы, например:

| Метод | Искажение | Repeatability (%) | Loc. error (px) | Inliers (%) | Time (s) | FPS |
|-------|-----------|-------------------|------------------|-------------|----------|-----|

3. Построить графики:
   - зависимость времени/кол-ва ключевых точек от разрешения;
   - зависимость метрик от силы искажения (например, длина размытия, уровень шума, степень сжатия).

In [ ]:
# Ваш код
# - Цикл по методам и искажениям
# - Вычисление метрик через описанные выше функции
# - Формирование таблиц (pandas или простые списки)
# - Построение графиков (matplotlib)

## **5. Задание 2. Мультимодальное сопоставление изображений**


### **5.1. Подбор мультимодальных пар**




**Варианты пар (минимум 2–3 типа):**

1. **RGB vs инфракрасные** (если найдёте открытые пары в интернете).
2. **День vs ночь** (одна и та же сцена).
3. **Спутниковые снимки vs карты** (Google Maps / OpenStreetMap / топографические карты).
4. **Исторические фото vs современные** (одна и та же улица, здание).

**Требование:**  
Для каждой пары кратко описать:
- источник изображений,
- основные отличия (контраст, шум, ракурс, сезон, освещение).


In [ ]:
# Ваш код
# Загрузка и визуализация выбранных пар изображений

---


**Что нужно сделать:**

- При необходимости привести изображения к одному размеру.
- Применить предобработку:
  - перевод в оттенки серого,
  - выравнивание гистограммы (`cv2.equalizeHist`),
  - CLAHE (адаптивное выравнивание `cv2.createCLAHE`) — особенно полезно для слабоконтрастных изображений.
- Сравнить, влияет ли предобработка на качество сопоставления.



In [ ]:
# Ваш код
# Функции:
# - preprocess_gray(image, use_clahe=False)
# - сравнение работы "до" и "после" предобработки

### **5.3. Сравнение SIFT vs ORB vs AKAZE для кросс-модальных задач**

**Что нужно сделать:**

1. Для каждой мультимодальной пары:
   - запустить пайплайн с SIFT,
   - отдельно с ORB,
   - отдельно с AKAZE.
2. Выполнить сопоставление дескрипторов (BFMatcher или FLANN).
3. При необходимости применить тест Лоу для отбраковки ложных срабатываний (особенно для SIFT/AKAZE).



In [ ]:
# Ваш код



---

### **5.4. Метрики: Precision/Recall, F1-score, ROC-AUC**




**Смысл:**

- Рассматриваем каждое сопоставление как бинарное решение:
  - **1 (true)** — правильное соответствие (инлайер),
  - **0 (false)** — неправильное соответствие (аутлайер).
- Имеем:
  - истинные метки (из маски RANSAC или из заранее подготовленной разметки),
  - некоторую «оценку качества» (например, расстояние между дескрипторами).

**Определения:**

- **Precision (точность)**:  
  $
  \text{Precision} = \frac{TP}{TP + FP}
  $
- **Recall (полнота)**:  
  $
  \text{Recall} = \frac{TP}{TP + FN}
  $
- **F1-score**:  
  $
  F1 = \frac{2 \cdot \text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}
  $
- **ROC-AUC** — площадь под ROC-кривой (TPR vs FPR при разных порогах).





```python
def compute_pr_f1_from_matches(distances, inlier_mask, threshold):
    """
    Вычисляет Precision, Recall и F1 для заданного порога расстояния.
    
    distances: np.array shape (N,) - расстояния между дескрипторами
               (например, m.distance из cv2.DMatch).
    inlier_mask: np.array shape (N,) - 1 для инлайеров, 0 для аутлайеров
                 (по результату RANSAC или разметки).
    threshold: порог по расстоянию, ниже которого считаем предсказание "пара является совпадением".
    """
    distances = np.array(distances, dtype=np.float32)
    y_true = np.array(inlier_mask).astype(int)  # 1 - правильное совпадение, 0 - ложное
    
    # Предсказание: 1 если расстояние <= threshold, иначе 0
    y_pred = (distances <= threshold).astype(int)
    
    TP = np.sum((y_pred == 1) & (y_true == 1))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    FN = np.sum((y_pred == 0) & (y_true == 1))
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall    = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    if precision + recall > 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = 0.0
    
    return precision, recall, f1, TP, FP, FN
```


In [ ]:
# Ваш код
# - получение distances и inlier_mask из сопоставлений
# - вычисление PR/F1 для нескольких порогов
# - построение графиков Precision–Recall и зависимости F1 от порога

### **5.5. Средняя ошибка репроекции**



**Смысл:**  
Похож на «точность локализации» из Задания 1, но теперь нас интересует, насколько хорошо гомография «переносятся» соответствующие точки с одного изображения на другое.

Можно использовать уже реализованную ранее функцию `compute_localization_error`, рассматривая среднюю ошибку как **среднюю ошибку репроекции**.




In [ ]:
# Ваш код
# - вычисление гомографии для мультимодальных пар
# - использование compute_localization_error для оценки


---

### **5.6. Анализ: влияние предобработки и выбор метода**



**Что нужно сделать:**

- Сравнить:
  - SIFT / ORB / AKAZE по метрикам PR, F1, ROC-AUC (при необходимости),
  - результаты до и после CLAHE / эквализации гистограммы.
- Сделать выводы:
  - какие методы лучше для кросс-модальных пар,
  - помогает ли предобработка и для каких типов изображений.




```python
# Ваш код
# - формирование таблиц и графиков для мультимодальных экспериментов
```

---


## **6. Задание 3. Детекторы для специфических доменов**



**ВЫБРАТЬ ОДИН ПУНКТ ПО ЖЕЛАНИЮ**

### **6.1. Медицинские изображения**


**Что нужно сделать:**

1. Подобрать несколько медицинских изображений (рентген, МРТ, УЗИ) одного и того же анатомического региона, но снятых:
   - в разное время,
   - с немного разным масштабом/углом,
   - либо с небольшими сдвигами.

2. Реализовать регистрацию (выравнивание) пары изображений с помощью ключевых точек:
   - детекция и дескрипторы (например, SIFT / ORB / AKAZE),
   - сопоставление,
   - оценка гомографии или аффинного преобразования,
   - выравнивание одного изображения под другое.

3. **Метрика:** точность регистрации изображений:
   - можно измерять среднюю ошибку репроекции на опорных точках (анатомические ориентиры),
   - либо визуально оценивать перекрытие (разница изображений до и после выравнивания).

In [ ]:
# Ваш код
# - сопоставление пар медицинских изображений через ключевые точки
# - использование compute_localization_error для оценки

---

### **6.2. Текстуры и паттерны**



**Что нужно сделать:**

1. Подобрать наборы текстур:
   - ткани (разные узоры),
   - природные текстуры (камень, дерево, трава),
   - техногенные (плитка, кирпич, асфальт и др.).

2. Задача: **оценить различимость похожих текстур**:
   - для каждой пары текстур (одинаковые/разные) вычислить ключевые точки и дескрипторы;
   - сопоставить дескрипторы между:
     - одинаковыми текстурами (разные кадры),
     - разными текстурами (по смыслу «не совпадают»).

3. Простая метрика различимости:
   - количество «хороших» совпадений (инлайеров) между:
     - `same_textures` — должно быть **высоким**,
     - `different_textures` — должно быть **низким**;
   - можно использовать отношение:
$$
R = \frac{\text{инлайеры между одинаковыми текстурами}}{\text{инлайеры между разными текстурами}}
$$

     Чем выше `R`, тем лучше алгоритм различает текстуры.

In [ ]:
# Ваш код
# - эксперименты с текстурами
# - подсчёт количества инлайеров для "похожих" и "непохожих" пар

### **6.3. Документы и текст (перспективные искажения и OCR)**



**Что нужно сделать:**

1. Подобрать изображения документов:
   - прямой скан/фото (почти без искажений),
   - тот же документ, снятый под углом (перспектива), с небольшими искажениями.

2. Задача:
   - по ключевым точкам оценить гомографию и выровнять перспективу документа (получить «квазискан»);
   - применить к выровненному изображению OCR (например, Tesseract, если есть возможность, либо любой другой распознаватель текста) — **не обязательно в коде, можно вручную/сторонним ПО**.

3. **Метрика:** качество OCR после выравнивания:
   - простой вариант: взять несколько строк текста и вручную посчитать **долю правильно распознанных символов**,
   - формула точности по символам:
$
\text{Accuracy} = 1 - \frac{\text{редакционное расстояние (Levenshtein)}}{\text{длина эталонного текста}}
$

Пример небольшой функции для подсчёта точности по строкам (при условии, что вы уже получили `gt_text` (эталонный текст) и `pred_text` (текст полученный OCR после выравнивания):






```python
def char_accuracy(gt_text, pred_text):
    """
    Простейшая оценка точности по символам:
    Accuracy = 1 - (редакционное расстояние / длина эталонного текста).
    Для учебных целей можно реализовать грубое расстояние Левенштейна
    или просто посчитать долю совпадающих символов на пересечении длин.
    """
    gt = str(gt_text)
    pr = str(pred_text)
    
    n = min(len(gt), len(pr))
    if n == 0:
        return 0.0
    
    matches = sum(1 for i in range(n) if gt[i] == pr[i])
    return matches / len(gt) * 100.0
```



*(Для серьёзной оценки лучше использовать полноценный алгоритм Левенштейна, но в рамках работы достаточно даже грубого приближения.)*

In [ ]:
# Ваш код
# - выравнивание документа по гомографии
# - передача результата в систему OCR (опционально)
# - расчёт простой метрики точности распознавания


---

## **7. Результаты**



Каждая работа должна содержать:

1. **Результаты**
   - Таблицы сравнения метрик для разных методов и условий:
     - для Задания 1 — по искажениям,
     - для Задания 2 — по мультимодальным парам,
     - для Задания 3 — по доменам,
   - Графики:
     - зависимость времени и количества ключевых точек от разрешения,
     - зависимость метрик от силы искажений (blur, noise, JPEG),
     - Precision–Recall / F1 от порога (для мультимодальных задач),
   - Визуализации:
     - ключевые точки на изображениях,
     - сопоставления (линии между изображениями),
     - карты инлайеров/аутлайеров,
     - результаты выравнивания (до/после гомографии, до/после коррекции перспективы).

In [ ]:
# Ваш код (ЕСЛИ ВСЕ ЭТО ОТСУТСТВУЕТ В ОТВЕТЕ НА КАЖДОЕ ЗАДАНИЕ)
# - формирование и вывод таблиц (pandas DataFrame или списки)
# - построение всех необходимых графиков
# - сохранение ключевых изображений с визуализацией


---

## **8. Анализ**

В разделе анализа рекомендуется:

1. Проверить **статистическую значимость различий**, где это оправдано:
   - сравнение средних значений метрик для разных методов (при многократных запусках).
2. Выполнить **корреляционный анализ метрик**:
   - как связаны время работы и качество (повторяемость, inlier ratio и т.п.);
   - как связаны разные метрики между собой (например, repeatability и процент инлайеров).
3. Выявить **закономерности**:
   - какие методы устойчивее к размытиям, шумам, сжатию,
   - какие комбинации лучше для кросс-модальных пар,
   - какие методы хорошо ведут себя на медицинских/текстурных/документальных изображениях,


**ВАШ ОТВЕТ**